In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

print('Setup complete')


# 1. Loading data from input directory...

In [2]:
# Path of the file to read
#submission_filepath = "../input/tabular-playground-series-apr-2021/sample_submission.csv"
#test_filepath = "../input/tabular-playground-series-apr-2021/test.csv"
train_filepath = "../input/tabular-playground-series-apr-2021/train.csv"

# Read the file into a variables 
#submission_data = pd.read_csv(submission_filepath, index_col="PassengerId", parse_dates=True)
#test_data = pd.read_csv(test_filepath, index_col="PassengerId", parse_dates=True)
train_data = pd.read_csv(train_filepath, index_col="PassengerId", parse_dates=True)
print('Data has been loaded')

# 2. Check data quality

2.1 Check columns info

In [ ]:
train_data.info()

In [27]:
train_data.describe()

2.1 Missing values

In [45]:
#lets see if there are any columns with missing values 
missing_values=train_data.isna().sum()
missing_values

In [ ]:
# visualization
plt.figure(figsize=(25,25))
missing_values = sns.displot(
    data=train_data.isna().melt(value_name="missing"),
    y="variable",
    hue="missing"
)

2.2 Unique values

In [ ]:
plt.figure(figsize=(14,8))
nu = train_data.nunique().reset_index()
nu.columns = ['columns','nunique']
sns.barplot(x='columns', y='nunique', data=nu)

2.3 Outlier

In [ ]:
sns.catplot(x='Sex',
            y='Age',
            kind='box',
            data=train_data);


My_notes: 
There are some warrings in data quality in file train.csv
1. there are empty cells in columns "Age", "Fare", "Ticket", "Cabin", "Embarked"
2. there are enough namesakes
3. all values are unique in column "PassengerId"

# 3. Clear data (delete not usefull data or add middle values data)

In [8]:
# drop useless columns
useless_columns = ['Name','Ticket']

train_clean = train_data.drop(columns=useless_columns)
#test_clean = test_data.drop(columns=useless_columns)
train_clean

In [5]:
def fill_data(df):
    
    df_copy = df.copy() #копируем, чтобы не изменять оригинал
    
    #for "object" type of data if NaN fill with mode
    categorial_vars = list(df_copy.select_dtypes(include=['object']).columns.values)
    for col in categorial_vars:
        
        if (df_copy[col].isnull().values.any()):
            df_copy[col].fillna(df_copy[col].mode()[0], inplace=True)
    
    
    #for numeric type of data if NaN fill with median
    numerical_vars = list(df_copy.select_dtypes(include=['int64', 'float64']).columns.values)
    for col in numerical_vars:        
        if (df_copy[col].isnull().values.any()):
            df_copy[col].fillna(df_copy[col].median(), inplace=True)

            
    return df_copy

In [9]:
# fill data with median and mode values
train_full = fill_data(train_clean)
#test_full = impute_data(test_clean)

# 4. Data Analyzing and visialization

3.1 Dependence between variables

In [10]:
#Add new colum 'sex_bool' with boolean values male-0, female-1
train_full['sex_bool']=np.where(train_full['Sex']=='male',0,1)
train_full

In [11]:
# Heatmap to see main correlations
plt.figure(figsize=(14,8))
corr = train_full.corr()
heatmap = sns.heatmap(corr, annot = True)


# My_note:
# 1. **Possitive correlation between Sex and Survived.** Its looks true, because women had opputinity to get into boats.
# 2. **Inverse correlation between Pclass and Fare.** Its looks true, because 1st class tickets are more expensive then # 3st class tickets
# 3. **Possitive correlation between Parch and Fare.** Its looks also true, because parents with there childen usually stay in one cabin and cabin for 3 person is more expensive then single cabin


In [12]:
# There were more poor men on the Titanic
sns.catplot('Pclass', data=train_full, hue='Sex', kind='count')

In [13]:
# Middle age people were the majority on the Titanic
sns.displot(train_full, x="Age", hue="Sex")

In [54]:
# dependence between Age and Survived
fig = plt.figure(figsize=(20,5))
plt.scatter(train_full.Age,train_full.Survived,  alpha=0.1)

In [14]:
# Dependence Sex and Pclass with Survived
sns.catplot('Sex', data=train_full, hue='Survived', kind='count');

sns.catplot('Pclass', data=train_full, hue='Survived', kind='count');

# My_note: Poor men had a little chance to survived on Titanic

In [23]:
# Accoding to correlation diagram upper there are no dependence betwin amount memeber of family and column survived
# but lets look closer
sns.catplot('SibSp', data=train_full, hue='Survived', kind='count');
# men with relatives
sns.catplot('SibSp', data=train_full[train_full['sex_bool'] == 0], hue='Survived', kind='count');
# woomen with relatives
sns.catplot('SibSp', data=train_full[train_full['sex_bool'] == 1], hue='Survived', kind='count');
sns.catplot('Parch', data=train_full, hue='Survived', kind='count');
# men with children
sns.catplot('Parch', data=train_full[train_full['sex_bool'] == 0], hue='Survived', kind='count');
# women with children
sns.catplot('Parch', data=train_full[train_full['sex_bool'] == 1], hue='Survived', kind='count');

# Single person on the Titanic had more chanse to survive

Note: I suppose that port as place where person got on the Titanic cannot influence on survival
But lets check dependence between Embarked and Sex or PClass. If the majority of passergers were poor men at some port and we have already known that there wene liitle chance to survive.

In [56]:
#The majority of passengers got on the Titatic in port S.
train_full.Embarked.value_counts().plot(kind='bar', alpha=0.55)

In [61]:
embarked_survived = train_full.groupby('Embarked').Survived.value_counts()
embarked_survived

In [62]:
# visualization
embarked_survived.unstack(level=0).plot(kind='bar', subplots=False)

# My_notes: The majority passengers got on the Titanic at port S. That is why it is not suprise that for port S survival rare is high, but port C is suprise for me. Lets look closer.

In [64]:
sex_embarked = train_full.groupby('Embarked').Sex.value_counts()
sex_embarked

# My_notes: There were much more female at port C, that is why port C has such good survival rate

In [66]:
sex_embarked = train_full.groupby('Embarked').Pclass.value_counts()
sex_embarked

# My_notes: There were much more 1st class passengers at port C, that is one more explanation why port C has such good survival rate